In [1]:
#conv Neural Network
# tensorboard --logdir=/home/ncc/notebook/learn/tensorboard/log
"""
created by kim Seong jung

RESNET MODULE 2016/2/7
CONVOLUTION MODULE 2016/2/6
batch_normalization 적용


"""
import numpy as np 
import tensorflow as tf
import re
import random
import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
import os 


# Load Image 

# load Training , Validation , Test 

In [2]:
in_ch =3
n_classes=2
img_row=128
img_col=128

In [3]:
with tf.device('/gpu:0'):
    def next_batch(batch_size , image , label):

        a=np.random.randint(np.shape(image)[0] -batch_size)
        batch_x = image[a:a+batch_size,:]
        batch_y= label[a:a+batch_size,:]
        return batch_x, batch_y


# define Variable , and placeholder 

In [4]:

x_= tf.placeholder("float",shape=[None,img_row , img_col , in_ch],  name = 'x-input')
y_= tf.placeholder("float",shape=[None , n_classes] , name = 'y-input')
keep_prob = tf.placeholder("float")
x_image= tf.reshape(x_,[-1,img_row,img_col,in_ch])



In [5]:

"""def weight_variable(name,shape):
    #initial = tf.truncated_normal(shape , stddev=0.1)
    initial = tf.get_variable(name,shape=shape , initializer = tf.contrib.layers.xavier_initializer())
    return tf.Variable(initial)"""
def bias_variable(shape , name):
    initial = tf.constant(0.1 , shape=shape , )
    return tf.Variable(initial , name=name)

def conv2d(x,w,strides_ , name):
    return tf.nn.conv2d(x,w, strides = strides_, padding='SAME' , name = name)

def max_pool(x , ksize , strides , padding , name ):
    return tf.nn.max_pool(x ,ksize , strides , padding ,name=name)
def make_weights_biases(layer_name , w_name , b_name,ksize ,device_name,initializer='xavier'):
    if len(ksize)==4: # convolution filter shape [batch , row , col , color_ch]
        out_ch=ksize[3]
    elif len(ksize)==2: #fully connected layer shape [in_ch , output_ch]
        out_ch=ksize[1]
    with tf.device(device_name):
        with tf.variable_scope(layer_name) as scope:
            try:
                w_conv = tf.get_variable(w_name, ksize , initializer = tf.contrib.layers.xavier_initializer())
            except:
                scope.reuse_variables()
                w_conv = tf.get_variable(w_name, ksize , initializer = tf.contrib.layers.xavier_initializer())        
            try:
                b_conv = bias_variable([out_ch] , name=b_name)
            except:
                scope.reuse_variables()
                b_conv = bias_variable([out_ch],name=b_name)
    return w_conv , b_conv 



# RESNET

In [6]:
def PRE_LAYER(x , device_,layer_name,n_nodes ):
    """
    
    """
    in_ch = x.get_shape()[3]
    out_ch=n_nodes    
    c_ksize=[7,7 , in_ch   , out_ch]
    c_pooling ='SAME'
    c_strides=[1,2,2,1] # when downsampling featrue map , set strides parameter,[1,2,2,1]
    w_conv , b_conv =make_weights_biases(layer_name, 'W' ,'B' , c_ksize ,device_name = device_)
    with tf.variable_scope(layer_name+'conv') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv , c_strides , c_pooling ,name=layer_name+'pre_conv')+b_conv
        layer1 = tf.nn.relu(layer1,name=layer_name+'relu')

    p_ksize=[1,3,3,1]
    p_strides=[1,2,2,1]
    
    layer2 = tf.nn.max_pool(layer1 ,p_ksize, c_strides , c_pooling , name=layer_name+'maxpool')
    return layer2


In [7]:
def LAYER_2(x , device_,layer_name,n_nodes , strides=[1,1,1,1]):
    in_ch = x.get_shape()[3]

    out_ch1=n_nodes
    out_ch2=n_nodes
    
    c_ksize1=[3,3 , in_ch   , out_ch1]
    c_ksize2=[3,3 , out_ch1 , out_ch2]
    
    c_strides1=strides # when downsampling featrue map , set strides parameter,[1,2,2,1]
    c_strides2=[1,1,1,1]
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1), 'W1' ,'B1' , c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2), 'W2' ,'B2' , c_ksize2 ,device_name = device_)
    c_pooling1='SAME'
    c_pooling2='SAME'
    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name=layer_name+'layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
        
    if x.get_shape()[1] != layer2.get_shape()[1] or x.get_shape()[2] != layer2.get_shape()[2]:
        pad_row=int((x.get_shape()[1]-layer2.get_shape()[1])/2)
        pad_col=int((x.get_shape()[2]-layer2.get_shape()[2])/2)
        #x_padded=tf.pad(x,paddings=[[pad_col,pad_col],[pad_row,pad_row]],mode="CONSTANT",name=layer_name+'x_padded')
        x_padded=tf.nn.max_pool(x,[1,2,2,1],[1,2,2,1],padding='SAME' , name=layer_name+'x_padded')
        
        x_ch=int(x_padded.get_shape()[3])
        print x_ch
        print x_padded.get_shape()
        w_conv , b_conv =make_weights_biases(layer_name, 'W' ,'B' , [1,1,x_ch,out_ch2] ,device_name = device_)
        x_conved = tf.nn.conv2d(x_padded,w_conv,strides=[1,1,1,1],padding="SAME",name=layer_name+'x_conved')+b_conv
        x_conved_relu =tf.nn.relu(x_conved,name=layer_name+'x_conved_relu')
        layer3=tf.add(layer2 ,x_conved_relu , name=layer_name+"end_add")
        layer3=tf.nn.relu(layer3 , name=layer_name+"end_relu")
        
    else:
        layer3=tf.add(layer2 ,x , name=layer_name+"end_add")
        layer3=tf.nn.relu(layer3 , name=layer_name+"end_relu")
    
    return layer3


In [8]:
def BOTTLENECT(x , device_,layer_name,n_nodes , strides=[1,1,1,1]):
    """
    n_nodes is tuple {}
    
    out_ch1=n_nodes[0]
    out_ch2=n_nodes[1]
    out_ch2=n_nodes[2]
    """
    
    in_ch = x.get_shape()[3]

    out_ch1=n_nodes[0]
    out_ch2=n_nodes[1]
    out_ch2=n_nodes[2]
    
    c_ksize1=[1,1 , in_ch   , out_ch1]
    c_ksize2=[3,3 , out_ch1 , out_ch2]
    c_ksize3=[1,1 , out_ch2 , out_ch3]
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1), 'W1' ,'B1' , c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2), 'W2' ,'B2' , c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3), 'W3' ,'B3' , c_ksize3 ,device_name = device_)
    
    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
    with tf.variable_scope(layer_name+'conv3') as scope:
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3 , name='layer3')+b_conv3
        layer3 = tf.nn.relu(layer3,name='layer3_relu')
    with tf.variable_scope(layer_name+'residual_end') as scope:
        layer4=tf.add(x,layer3 , layer_name+'add')
        layer4=tf.nn.relu(layer4 , name=layer_name+'relu') 
    return layer4


# INCEPTION_NET_V4

In [9]:
def STEM_A(x , device_):
    with tf.device(device_):
        in_ch=x.get_shape()[3]
        out_ch1=32 ; out_ch2=32;out_ch3=64;out_ch4=96;

        
        c_ksize1=[3,3,in_ch , out_ch1]
        c_ksize2=[3,3,out_ch1 , out_ch2]
        c_ksize3=[3,3,out_ch2 , out_ch3]
        c_ksize4=[3,3,out_ch3 , out_ch4]
        w_conv1 , b_conv1 =make_weights_biases('STEM_A_WB_1' , 'W1' ,'B1', c_ksize1 ,device_name = device_)
        w_conv2 , b_conv2= make_weights_biases('STEM_A_WB_2' , 'W2' ,'B2',c_ksize2 ,device_name = device_)
        w_conv3 , b_conv3= make_weights_biases('STEM_A_WB_3' , 'W3' ,'B3' ,c_ksize3 ,device_name = device_)
        w_conv4 , b_conv4= make_weights_biases('STEM_A_WB_4' , 'W4' ,'B4', c_ksize4 ,device_name = device_)

        
        c_strides1=[1,2,2,1]
        c_strides2=[1,1,1,1]
        c_strides3=[1,1,1,1]
        c_strides4=[1,2,2,1]

        
        c_pooling1='VALID'
        c_pooling2='VALID'
        c_pooling3='SAME'
        c_pooling4='VALID'

        b_p_ksize4=[1,3,3,1]
        b_p_strides4=[1,2,2,1]
        b_p_padding4 ='VALID'

        with tf.variable_scope('STEM_A_layer1') as scope:
            layer1     =tf.nn.conv2d(    x ,      w_conv1, c_strides1, c_pooling1 ,name='conv2d'  )+b_conv1
            layer1_relu=tf.nn.relu(layer1,name='relu')
        with tf.variable_scope('STEM_A_layer2') as scope:
            layer2     =tf.nn.conv2d(    layer1_relu , w_conv2, c_strides2, c_pooling2,name='conv2d'  )+b_conv2
            layer2_relu=tf.nn.relu(layer2,name='relu')
        with tf.variable_scope('STEM_A_layer3') as scope:
            layer3     =tf.nn.conv2d(    layer2_relu , w_conv3, c_strides3, c_pooling3,name='conv2d'  )+b_conv3
            layer3_relu=tf.nn.relu(layer3 , name='relu')
        with tf.variable_scope('STEM_A_layer4') as scope:
            layer4     =tf.nn.conv2d(    layer3_relu , w_conv4, c_strides4, c_pooling4 ,name='conv2d' )+b_conv4
            layer4_relu=tf.nn.relu(layer4,name='relu')
      
        with tf.variable_scope('STEM_A_b_layer') as scope:
            b_layer4=tf.nn.max_pool(layer3 , b_p_ksize4, b_p_strides4, b_p_padding4 ,name='max_pool') #b is branch
            b_layer4_relu=tf.nn.relu(b_layer4 , name = 'relu')
        print layer1
        print layer2
        print layer3
        print layer4
        print b_layer4


        concat_layer=tf.concat(3 , [layer4_relu , b_layer4_relu] , name='CONCAT')
        return concat_layer

In [10]:
def STEM_B( x , device_):
    with tf.device(device_):

        in_ch=x.get_shape()[3]
        #########################################################################
        out_ch1=64 ; out_ch2=64;out_ch3=64;out_ch4=96;
        ##############################right side##################################
        c_ksize1=[1,1,in_ch , out_ch1]
        c_ksize2=[7,1,out_ch1 , out_ch2]
        c_ksize3=[1,7,out_ch2 , out_ch3]
        c_ksize4=[3,3,out_ch3 , out_ch4]

        w_conv1 , b_conv1 =make_weights_biases('STEM_B_WB_1' , 'W1' ,'B1', c_ksize1 ,device_name = device_)
        w_conv2 , b_conv2= make_weights_biases('STEM_B_WB_2' , 'W2' ,'B2',c_ksize2 ,device_name = device_)
        w_conv3 , b_conv3= make_weights_biases('STEM_B_WB_3' , 'W3' ,'B3' ,c_ksize3 ,device_name = device_)
        w_conv4 , b_conv4= make_weights_biases('STEM_B_WB_4' , 'W4' ,'B4', c_ksize4 ,device_name = device_)


        c_strides1=[1,1,1,1]
        c_strides2=[1,1,1,1]
        c_strides3=[1,1,1,1]
        c_strides4=[1,1,1,1]

        c_pooling1='SAME'
        c_pooling2='SAME'
        c_pooling3='SAME'
        c_pooling4='VALID'
        
        with tf.variable_scope('STEM_B_layer1') as scope:
            layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='conv2d')+b_conv1
            layer1_relu = tf.nn.relu(layer1,name='relu')
        with tf.variable_scope('STEM_B_layer2') as scope:
            layer2 = tf.nn.conv2d(layer1_relu , w_conv2 , c_strides2 , c_pooling2 , name='conv2d')+b_conv2
            layer2_relu = tf.nn.relu(layer2,name='relu')
        with tf.variable_scope('STEM_B_layer3') as scope:
            layer3 = tf.nn.conv2d(layer2_relu , w_conv3 , c_strides3 , c_pooling3,name='conv2d')+b_conv3
            layer3_relu = tf.nn.relu(layer3,name='relu')
        with tf.variable_scope('STEM_B_layer4') as scope:
            layer4 = tf.nn.conv2d(layer3_relu , w_conv4 , c_strides4 , c_pooling4,name='conv2d')+b_conv4
            layer4_relu = tf.nn.relu(layer4,name='relu')

        ##############################left side##################################
        b_out_ch1=64 ; b_out_ch2=96;    
        #########################################################################
        b_c_ksize1=[1,1,in_ch , b_out_ch1]
        b_c_ksize2=[3,3,b_out_ch1 , b_out_ch2]
 
        b_w_conv1 , b_b_conv1 =make_weights_biases('STEM_B_b_1' , 'b_W1' , 'b_B1',b_c_ksize1 ,device_name = device_)
        b_w_conv2 , b_b_conv2= make_weights_biases('STEM_B_b_2' , 'b_W2' , 'b_B2',b_c_ksize2 ,device_name = device_)

        b_c_strides1=[1,1,1,1]
        b_c_strides2=[1,1,1,1]

        b_c_pooling1='SAME'
        b_c_pooling2='VALID'
        with tf.variable_scope('STEM_B_b_layer1') as scope:
            b_layer1      =tf.nn.conv2d(    x ,      b_w_conv1, b_c_strides1, b_c_pooling1,name='conv2d'  ) + b_b_conv1
            b_layer1_relu = tf.nn.relu(b_layer1,name='relu')
        with tf.variable_scope('STEM_B_b_layer2') as scope:
            b_layer2     =tf.nn.conv2d(  b_layer1_relu , b_w_conv2, b_c_strides2, b_c_pooling2,name='conv2d'  ) + b_b_conv2
            b_layer2_relu= tf.nn.relu(b_layer2 , name='relu')
        ##############################concatenate layers###########################
        concat_layer=tf.concat(3 , [layer4_relu , b_layer2_relu] , name="STEM_B_CONCAT")
        ret_layer=concat_layer
    return ret_layer

In [11]:
def STEM_C( x , device_  ):
    with tf.device(device_):
        in_ch=x.get_shape()[3]
        #########################################################################

        out_ch = 192


        c_ksize=[3,3,in_ch,out_ch]
        w_conv , b_conv =make_weights_biases('STEM_C_WB' , 'W' , 'B',c_ksize ,device_name = device_)
        c_strides=[1,1,1,1]
        c_pooling='SAME'
        with tf.variable_scope('STEM_C_layer') as scope:
            layer = tf.nn.conv2d(x , w_conv   , c_strides   , c_pooling , name='conv2d')+b_conv
            layer_relu = tf.nn.relu(layer,name='relu')

        #########################################################################
        
        b_p_ksize=[1,2,2,1]
        b_p_strides=[1,1,1,1]
        b_p_pooling='SAME'
        with tf.variable_scope('STEM_C_b_layer') as scope:
            b_layer = tf.nn.max_pool(x , b_p_ksize , b_p_strides , b_p_pooling,name='max_pool' )
            b_layer_relu = tf.nn.relu(b_layer,name='relu')
        #########################################################################

        print layer
        print b_layer

        concat_layer=tf.concat(3,[layer_relu , b_layer_relu],name='STEM_C_concat')
        ret_layer = concat_layer
    return ret_layer

In [12]:
def INCEPTION_MODULE_A(x , device_ , layer_name = 'INCEPTION_MODULE_A'):

    """
    input X shape is [n_batch , row , col, out_ch]
    35 x 35 grid
    """

    #################################################################################
    in_ch=x.get_shape()[3]
    print x.get_shape()
    out_ch1=64 ; out_ch2= 96;
    c_ksize1 = [1 , 1 ,in_ch , out_ch1 ]
    c_ksize2 = [3 , 3 ,out_ch1 , out_ch2 ]
    w_conv1 , b_conv1 = make_weights_biases(layer_name+'WB_1', 'W1', 'B1',c_ksize1 ,device_)
    w_conv2 , b_conv2 = make_weights_biases(layer_name+'WB_2' ,'W2' ,'B2', c_ksize2 ,device_)

    c_strides1=[1,1,1,1]
    c_strides2=[1,1,1,1]

    c_pooling1='SAME'
    c_pooling2='SAME'
    with tf.variable_scope(layer_name+'layer1') as scope:
        layer1 = tf.nn.conv2d(x , w_conv1 , c_strides1 ,c_pooling1  ,name= 'conv2d')
        layer1_relu=tf.nn.relu(layer1,name='relu')
    with tf.variable_scope(layer_name+'layer2') as scope:
        layer2 = tf.nn.conv2d(layer1_relu , w_conv2 , c_strides2 ,c_pooling2 ,name='conv2d')        
        layer2_relu = tf.nn.relu(layer2 , name = 'relu')

    #################################################################################                     

    b1_p_ksize   =[1,2,2,1]
    b1_p_strides =[1,1,1,1]
    b1_p_pooling='SAME'


    b1_out_ch = 96;
    b1_c_ksize  =[1,1, in_ch , b1_out_ch]
    b1_c_strides=[1,1,1,1]
    b1_c_pooling ='SAME'

    b1_w_conv , b1_b_conv =make_weights_biases(layer_name+'b1_WB','b1_W','b1_B',b1_c_ksize , device_)

    with tf.variable_scope(layer_name+'b1_layer1') as scope:
        b1_layer1      =tf.nn.avg_pool(x, b1_p_ksize , b1_p_strides,b1_p_pooling ,name='avg_pool')
        b1_layer1_relu =tf.nn.relu(b1_layer1,name='relu')
                
    with tf.variable_scope(layer_name+'b1_layer2') as scope:
        b1_layer2      =tf.nn.conv2d(b1_layer1_relu , b1_w_conv , b1_c_strides , b1_c_pooling, name ='conv2d')+b1_b_conv
        b1_layer2_relu =tf.nn.relu(b1_layer2 , name ='relu')

    #################################################################################                     

    b2_out_ch=96
    b2_c_ksize = [1,1,in_ch , b2_out_ch ]
    b2_w_conv , b2_b_conv = make_weights_biases(layer_name+'b2_WB','b2_W','b1_B',b2_c_ksize , device_)
    b2_c_strides=[1,1,1,1]
    b2_c_pooling='SAME'
    with tf.variable_scope(layer_name+'b2_layers') as scope:        
        b2_layer      = tf.nn.conv2d( x, b2_w_conv , b2_c_strides ,b2_c_pooling ,name='conv2d') +b2_b_conv 
        b2_layer_relu = tf.nn.relu(b2_layer , name='relu')



    #################################################################################                     


    b3_out_ch1=64;b3_out_ch2=96;b3_out_ch3=96;
    b3_c_ksize1 =[1,1,in_ch , b3_out_ch1]
    b3_c_ksize2 =[3,3,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3 =[3,3,b3_out_ch2 , b3_out_ch3]
    b3_w_conv1 , b3_b_conv1 = make_weights_biases(layer_name+'b3_WB_1','b3_W1','b3_B1',b3_c_ksize1 , device_)
    b3_w_conv2 , b3_b_conv2 = make_weights_biases(layer_name+'b3_WB_2','b3_W2','b3_B2',b3_c_ksize2 , device_)
    b3_w_conv3 , b3_b_conv3 = make_weights_biases(layer_name+'b3_WB_3','b3_W3','b3_B3',b3_c_ksize3 , device_)
    b3_c_strides1=[1,1,1,1]
    b3_c_strides2=[1,1,1,1]
    b3_c_strides3=[1,1,1,1]
    b3_c_pooling1='SAME'
    b3_c_pooling2='SAME'
    b3_c_pooling3='SAME'
    with tf.variable_scope(layer_name + 'b3_layer1') as scope:
        b3_layer1 = tf.nn.conv2d(x , b3_w_conv1 , b3_c_strides1 ,b3_c_pooling1,name='conv2d')+b3_b_conv1 
        b3_layer1_relu = tf.nn.relu(b3_layer1,name='relu')
    with tf.variable_scope(layer_name + 'b3_layer2') as scope:
        b3_layer2 = tf.nn.conv2d(b3_layer1_relu , b3_w_conv2 , b3_c_strides2 ,b3_c_pooling2,name='conv2d')+b3_b_conv2 
        b3_layer2_relu= tf.nn.relu(b3_layer2,name='conv2d')
    with tf.variable_scope(layer_name + 'b3_layer3') as scope:
        b3_layer3 = tf.nn.conv2d(b3_layer2_relu , b3_w_conv3 , b3_c_strides3 ,b3_c_pooling3 , name='conv2d')+b3_b_conv3 
        b3_layer3_relu = tf.nn.relu(b3_layer3,name='relu')

    #################################################################################                     
    with tf.variable_scope(layer_name+'concat') as scope:
        concat_A=tf.concat(3,[layer2_relu , b1_layer2_relu] , name = '1')
        concat_B=tf.concat(3,[concat_A , b2_layer_relu],name = '2')
        concat_C=tf.concat(3,[concat_B , b3_layer3_relu],name = '3')

    return concat_C

    #################################################################################                     





In [13]:
def INCEPTION_MODULE_B(x , device ,layer_name='INCEPTION_MODULE_B'):

    """
    for 17 X 17 grid 
    """
    in_ch =x.get_shape()[3]
    out_ch1 =192 ; out_ch2=224; out_ch3 =256
    c_ksize1 = [1,1, in_ch  ,out_ch1]
    c_ksize2 = [1,7, out_ch1,out_ch2]
    c_ksize3 = [7,1, out_ch2,out_ch3]

    w_conv1 , b_conv1 =make_weights_biases( layer_name+'WB_1','W1'  ,'B1',  c_ksize1 , device)
    w_conv2 , b_conv2 =make_weights_biases( layer_name+'WB_2' ,'W2' ,'B2', c_ksize2 , device)
    w_conv3 , b_conv3 =make_weights_biases( layer_name+'WB_3','W3' ,'B3',  c_ksize3 , device)
    
    c_strides1 =[1,1,1,1]
    c_strides2 =[1,1,1,1]
    c_strides3 =[1,1,1,1]
    
    c_pooling1 ='SAME'
    c_pooling2 ='SAME'
    c_pooling3 ='SAME'

    with tf.variable_scope(layer_name+'layer1') as scope:
        layer1     = tf.nn.conv2d(x,      w_conv1,c_strides1 , c_pooling1 ) +b_conv1
        layer1_relu = tf.nn.relu(layer1)
    with tf.variable_scope(layer_name+'layer2') as scope:
        layer2      = tf.nn.conv2d(layer1_relu, w_conv2,c_strides2 , c_pooling2 ) +b_conv2
        layer2_relu = tf.nn.relu(layer2)
    with tf.variable_scope(layer_name+'layer3') as scope:
        layer3      = tf.nn.conv2d(layer2_relu, w_conv3,c_strides3 , c_pooling3 ) +b_conv3
        layer3_relu = tf.nn.relu(layer3)


    ################################################################################# 


    b1_p_ksize=[1,2,2,1]
    b1_p_strides=[1,1,1,1]
    b1_p_pooling ='SAME'

    b1_out_ch = 128
    b1_c_ksize=[1,1,in_ch,b1_out_ch]
    b1_w_conv , b1_b_conv = make_weights_biases(layer_name+'b1_WB_1','b1_W1','b1_B1',b1_c_ksize, device )
    b1_c_strides=[1,1,1,1]
    b1_c_pooling='SAME'

    with tf.variable_scope(layer_name+'b1_layer1') as scope:
        b1_layer1      =tf.nn.avg_pool(x , b1_p_ksize , b1_p_strides , b1_p_pooling ,name='conv')
        b1_layer1_relu =tf.nn.relu(b1_layer1,name='relu')
    with tf.variable_scope(layer_name+'b1_layer2') as scope:
        b1_layer2      =tf.nn.conv2d(b1_layer1_relu ,b1_w_conv , b1_c_strides, b1_c_pooling ,name='conv')+b1_b_conv 
        b1_layer2_relu =tf.nn.relu(b1_layer2,name='relus')

    ################################################################################# 
    b2_out_ch=384
    b2_c_ksize=[1,1,in_ch,b2_out_ch]
    b2_w_conv , b2_b_conv =make_weights_biases(layer_name+'b2_WB_1','b2_W1','b1_B1',b2_c_ksize , device)
    b2_c_strides=[1,1,1,1]
    b2_c_pooling='SAME'
    with tf.variable_scope(layer_name+'b2_layer') as scope:
        b2_layer      = tf.nn.conv2d(x,b2_w_conv , b2_c_strides , b2_c_pooling , name='relus')+b2_b_conv
        b2_layer_relu = tf.nn.relu(b2_layer,name='relu')
    ################################################################################# 


    b3_out_ch1=192; b3_out_ch2 =192; b3_out_ch3=224 ; b3_out_ch4=224 ; b3_out_ch5 =256
    b3_c_ksize1=[1,1,in_ch,b3_out_ch1]
    b3_c_ksize2=[1,7,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3=[7,1,b3_out_ch2 , b3_out_ch3]
    b3_c_ksize4=[1,7,b3_out_ch3 , b3_out_ch4]
    b3_c_ksize5=[7,1,b3_out_ch4 , b3_out_ch5]

    b3_w_conv1 , b3_b_conv1=make_weights_biases(layer_name+'b3_WB_1','b3_W1','b3_B1',b3_c_ksize1, device)
    b3_w_conv2 , b3_b_conv2=make_weights_biases(layer_name+'b3_WB_2','b3_W2','b3_B2',b3_c_ksize2, device)
    b3_w_conv3 , b3_b_conv3=make_weights_biases(layer_name+'b3_WB_3','b3_W3','b3_B3',b3_c_ksize3, device)
    b3_w_conv4 , b3_b_conv4=make_weights_biases(layer_name+'b3_WB_4','b3_W4','b3_B4',b3_c_ksize4, device)
    b3_w_conv5 , b3_b_conv5=make_weights_biases(layer_name+'b3_WB_5','b3_W5','b3_B5',b3_c_ksize5, device)
    
    b3_c_strides_1=[1,1,1,1]
    b3_c_strides_2=[1,1,1,1]
    b3_c_strides_3=[1,1,1,1]
    b3_c_strides_4=[1,1,1,1]
    b3_c_strides_5=[1,1,1,1]
    
    b3_c_pooling_1="SAME"
    b3_c_pooling_2="SAME"
    b3_c_pooling_3="SAME"
    b3_c_pooling_4="SAME"
    b3_c_pooling_5="SAME"
    
    with tf.variable_scope(layer_name+'b3_layer1') as scope:
        b3_layer1 = tf.nn.conv2d(x,b3_w_conv1 , b3_c_strides_1 , b3_c_pooling_1 , name='conv2d')+b3_b_conv1
        b3_layer1_relu = tf.nn.relu(b3_layer1 , name='relu')
    with tf.variable_scope(layer_name+'b3_layer2') as scope:

        b3_layer2 = tf.nn.conv2d(b3_layer1_relu,b3_w_conv2 , b3_c_strides_2 , b3_c_pooling_2,name='conv2d')+b3_b_conv2
        b3_layer2_relu = tf.nn.relu(b3_layer2,name='relu')
    with tf.variable_scope(layer_name+'b3_layer3') as scope:

        b3_layer3 = tf.nn.conv2d(b3_layer2_relu,b3_w_conv3 , b3_c_strides_3 , b3_c_pooling_3,name='conv2d')+b3_b_conv3
        b3_layer3_relu = tf.nn.relu(b3_layer3,name='relu')
    with tf.variable_scope(layer_name+'b3_layer4') as scope:

        b3_layer4 = tf.nn.conv2d(b3_layer3_relu,b3_w_conv4 , b3_c_strides_4 , b3_c_pooling_4,name='conv2d')+b3_b_conv4
        b3_layer4_relu = tf.nn.relu(b3_layer4,name='relu')
    with tf.variable_scope(layer_name+'b3_layer5') as scope:

        b3_layer5 = tf.nn.conv2d(b3_layer4_relu,b3_w_conv5 , b3_c_strides_5 , b3_c_pooling_5,name='conv2d')+b3_b_conv5
        b3_layer5_relu = tf.nn.relu(b3_layer5,name='relu')

    ################################################################################# 

    with tf.variable_scope(layer_name+'concat') as scope:
        layerA=tf.concat(3, [layer3_relu , b1_layer2_relu] ,name='1')
        layerB=tf.concat(3, [layerA , b2_layer_relu] ,name='2')
        layerC=tf.concat(3, [layerB , b3_layer5_relu] ,name='3')

    print layerC
    return layerC


In [14]:
def INCEPTION_MODULE_C(x , device , layer_name='INCEPTION_MODULE_C'):

    """
    for 8 X 8 grid modules
    """
    print x.get_shape()
    in_ch=x.get_shape()[3]
    
    out_ch1 = 384 ; out_ch2_a =256 ;out_ch2_b = 256
    c_ksize1 = [1,1,in_ch , out_ch1]
    c_ksize2_a = [1,3,out_ch1 , out_ch2_a]
    c_ksize2_b = [3,1,out_ch1 , out_ch2_b]
    w_conv1 ,b_conv1=make_weights_biases(layer_name+'WB_1','W1','B1', c_ksize1 ,device)
    w_conv2_a ,b_conv2_a=make_weights_biases(layer_name+'WB_2_a','W2_a','B2_a', c_ksize2_a ,device)
    w_conv2_b ,b_conv2_b=make_weights_biases(layer_name+'WB_2_b','W2_b','B2_b', c_ksize2_b ,device)
    c_strides1=[1,1,1,1]
    c_strides2_a=[1,1,1,1]
    c_strides2_b=[1,1,1,1]
    c_pooling1='SAME'
    c_pooling2_a='SAME'
    c_pooling2_b='SAME'
    with tf.variable_scope(layer_name+'layer1') as scope:
        layer1     = tf.nn.conv2d(x, w_conv1 ,c_strides1 ,c_pooling1 , name='conv2d')+b_conv1
        layer1_relu=tf.nn.relu(layer1 , name= 'relu')
    with tf.variable_scope(layer_name+'layer2_a') as scope:
        layer2_a     = tf.nn.conv2d(layer1_relu,  w_conv2_a ,c_strides2_a ,c_pooling2_a ,name='conv2d')+b_conv2_a
        layer2_a_relu= tf.nn.relu(layer2_a,name='relu')
    with tf.variable_scope(layer_name+'layer2_b') as scope:
        layer2_b      = tf.nn.conv2d(layer1_relu  , w_conv2_a ,c_strides2_a ,c_pooling2_b,name='conv2d')+b_conv2_b
        layer2_b_relu = tf.nn.relu(layer2_b,name='relu')
    ################################################################################# 


    b1_p_ksize=[1,2,2,1]
    b1_p_strides=[1,1,1,1]
    b1_p_pooling='SAME'

    b1_out_ch =256
    b1_c_ksize=[1,1,in_ch , b1_out_ch]
    b1_w_conv , b1_b_conv= make_weights_biases(layer_name+'b1_WB', 'b1_W1' ,'b1_B1', b1_c_ksize , device)
    b1_c_pooling = 'SAME';b1_c_strides = [1,1,1,1];
    with tf.variable_scope(layer_name+'b1_layer1') as scope:    
        b1_layer1     = tf.nn.avg_pool(x , b1_p_ksize , b1_p_strides , b1_p_pooling , name='avg_pool')
        b1_layer1_relu= tf.nn.relu(b1_layer1 , name='relu')
    with tf.variable_scope(layer_name+'b1_layer2') as scope:    
        b1_layer2      = tf.nn.conv2d(b1_layer1_relu, b1_w_conv , b1_c_strides  ,b1_c_pooling , name ='conv2d')+b1_b_conv
        b1_layer2_relu = tf.nn.relu(b1_layer2,name='relu')
    ################################################################################# 
    b2_out_ch=256
    b2_c_ksize=[1,1,in_ch,b2_out_ch]
    b2_w_conv , b2_b_conv= make_weights_biases(layer_name+'b2_WB' , 'b2_W' , 'b1_B',b2_c_ksize , device)
    b2_c_pooling = 'SAME';b2_c_strides=[1,1,1,1];
    with tf.variable_scope(layer_name+'b2_layer') as scope:
        b2_layer     = tf.nn.conv2d(x , b2_w_conv , b2_c_strides  ,b2_c_pooling,name='conv2d')+b2_b_conv
        b2_layer_relu= tf.nn.relu(b2_layer ,name='relu')
   
    ################################################################################# 


    b3_out_ch1=384;b3_out_ch2=448;b3_out_ch3=512;b3_out_ch4_a=256;b3_out_ch4_b=256
    b3_c_ksize1 =[1,1,in_ch , b3_out_ch1]
    b3_c_ksize2 =[1,3,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3 =[3,1,b3_out_ch2 , b3_out_ch3]
    b3_c_ksize4_a =[3,1,b3_out_ch3 , b3_out_ch4_a]
    b3_c_ksize4_b =[1,3,b3_out_ch3, b3_out_ch4_b]

    b3_w_conv1 , b3_b_conv1 = make_weights_biases(layer_name+'b3_WB_1','b3_W1','b3_B1',b3_c_ksize1 , device)
    b3_w_conv2 , b3_b_conv2 = make_weights_biases(layer_name+'b3_WB_2','b3_W2','b3_B2',b3_c_ksize2 , device)
    b3_w_conv3 , b3_b_conv3 = make_weights_biases(layer_name+'b3_WB_3','b3_W3','b3_B3',b3_c_ksize3 , device)
    b3_w_conv4_a , b3_b_conv4_a = make_weights_biases(layer_name+'b3_WB_4_a','b3_W4_a','b3_B4_a',b3_c_ksize4_a , device)
    b3_w_conv4_b , b3_b_conv4_b = make_weights_biases(layer_name+'b3_WB_4_b','b3_W4_b','b3_B4_b',b3_c_ksize4_b , device)

    b3_c_strides1=[1,1,1,1]
    b3_c_strides2=[1,1,1,1]
    b3_c_strides3=[1,1,1,1]
    b3_c_strides4_a=[1,1,1,1]
    b3_c_strides4_b=[1,1,1,1]

    b3_c_pooling1='SAME'
    b3_c_pooling2='SAME'
    b3_c_pooling3='SAME'
    b3_c_pooling4_a='SAME'
    b3_c_pooling4_b='SAME'
    with tf.variable_scope(layer_name+'b3_layer1') as scope:    
        b3_layer1       = tf.nn.conv2d(x , b3_w_conv1 , b3_c_strides1 ,b3_c_pooling1)+b3_b_conv1 
        b3_layer1_relu = tf.nn.relu(b3_layer1)
    with tf.variable_scope(layer_name+'b3_layer2') as scope:    
        b3_layer2      = tf.nn.conv2d(b3_layer1_relu , b3_w_conv2 , b3_c_strides2 ,b3_c_pooling2)+b3_b_conv2 
        b3_layer2_relu = tf.nn.relu(b3_layer2)
    with tf.variable_scope(layer_name+'b3_layer3') as scope:    
        b3_layer3      = tf.nn.conv2d(b3_layer2_relu , b3_w_conv3 , b3_c_strides3 ,b3_c_pooling3)+b3_b_conv3 
        b3_layer3_relu = tf.nn.relu(b3_layer3)
    with tf.variable_scope(layer_name+'b3_layer4') as scope:    
        b3_layer4_a      = tf.nn.conv2d(b3_layer3_relu , b3_w_conv4_a , b3_c_strides4_a ,b3_c_pooling4_a)+b3_b_conv4_a 
        b3_layer4_a_relu = tf.nn.relu(b3_layer4_a)
    with tf.variable_scope(layer_name+'b3_layer5') as scope:    
        b3_layer4_b      = tf.nn.conv2d(b3_layer3_relu , b3_w_conv4_b , b3_c_strides4_b ,b3_c_pooling4_b)+b3_b_conv4_b 
        b3_layer4_b_relu = tf.nn.relu(b3_layer4_b)


    #################################################################################  with tf.variable_scope(layer_name+'b3_layer5') as scope:    
    with tf.variable_scope(layer_name+'concat') as scope:    
        layerA = tf.concat(3 , [layer2_a_relu, layer2_b_relu],name='1')
        layerB = tf.concat(3 , [layerA  , b1_layer2_relu],name='2')
        layerC = tf.concat(3 , [layerB  , b2_layer_relu],name='3')
        layerD = tf.concat(3 , [layerC  , b3_layer4_a_relu],name='4')
        layerE = tf.concat(3 , [layerD  , b3_layer4_b_relu],name='5')

    return layerE

In [15]:
def INCEPTION_REDUCTION_A(x , device , layer_name = 'INCEPTION_REDUCTION_A'):
    ####################################################################################
   
    """
    usage:
    x shape =[ n_batch , row , col , ch] 
    """
    print x.get_shape()
    in_ch=x.get_shape()[3]
    out_ch = 384
    c_ksize= [3,3,in_ch , out_ch]
    w_conv , b_conv =make_weights_biases(layer_name+"WB",'W','B',c_ksize,device)
    c_strides=[1,2,2,1]
    c_pooling='VALID'
    
    with tf.variable_scope(layer_name+'layer') as scope:    
        layer=tf.nn.conv2d(x,w_conv,c_strides , c_pooling , name='conv2d')+b_conv
        layer_relu=tf.nn.relu(layer , name='relu')

    ####################################################################################
   
    b1_p_ksize=[1,3,3,1]
    b1_p_strides=[1,2,2,1]
    b1_p_pooling='VALID'
    with tf.variable_scope(layer_name+'b1_layer') as scope:
        b1_layer = tf.nn.max_pool(x,b1_p_ksize , b1_p_strides , b1_p_pooling, name='max_pool')
        b1_layer_relu = tf.nn.relu(b1_layer , name='relu')
    ####################################################################################
   
    b2_out_ch1 = 192; b2_out_ch2=288 ;b2_out_ch3 = 256;
    b2_c_ksize1=[1,1,in_ch,b2_out_ch1]
    b2_c_ksize2=[3,3,b2_out_ch1,b2_out_ch2]
    b2_c_ksize3=[3,3,b2_out_ch2,b2_out_ch3]
    b2_w_conv1 , b2_b_conv1 = make_weights_biases(layer_name+'b2_WB_1' ,'b2_W1' ,'b2_B1', b2_c_ksize1 , device)
    b2_w_conv2 , b2_b_conv2 = make_weights_biases(layer_name+'b2_WB_2' ,'b2_W2' ,'b2_B2',b2_c_ksize2 , device)
    b2_w_conv3 , b2_b_conv3 = make_weights_biases(layer_name+'b2_WB_3' ,'b2_W3' ,'b2_B3',b2_c_ksize3 , device)
    
    b2_c_strides1=[1,1,1,1]
    b2_c_strides2=[1,1,1,1]
    b2_c_strides3=[1,2,2,1]
    b2_c_pooling1='SAME'
    b2_c_pooling2='SAME'
    b2_c_pooling3='VALID'
    with tf.variable_scope(layer_name+'b2_layer1') as scope:    
        b2_layer1      = tf.nn.conv2d(x        , b2_w_conv1 ,b2_c_strides1 ,b2_c_pooling1 , name='conv2d')+b2_b_conv1
        b2_layer1_relu = tf.nn.relu(b2_layer1,name='relu')
    with tf.variable_scope(layer_name+'b2_layer2') as scope:
        b2_layer2     = tf.nn.conv2d(b2_layer1_relu, b2_w_conv2 ,b2_c_strides2 ,b2_c_pooling2,name='conv2d')+b2_b_conv2
        b2_layer2_relu= tf.nn.relu(b2_layer2,name='relu')
    with tf.variable_scope(layer_name+'b2_layer3') as scope:
        b2_layer3      = tf.nn.conv2d(b2_layer2_relu, b2_w_conv3 ,b2_c_strides3 ,b2_c_pooling3,name='conv2d')+b2_b_conv3
        b2_layer3_relu = tf.nn.relu(b2_layer3,name='relu')
    
    print layer.get_shape()
    print b1_layer.get_shape()
    print b2_layer3.get_shape()
    ####################################################################################
    with tf.variable_scope(layer_name+'concat') as scope:    
        layerA=tf.concat(3 ,[layer_relu ,b1_layer_relu ] ,name='1')
        layerB=tf.concat(3 ,[layerA ,b2_layer3_relu],name='2')

    
    
    return layerB
    

In [16]:
def INCEPTION_REDUCTION_B(x , device  , layer_name = 'INCEPTION_REDUCTION_B'):
    """
    
    """
    
    ####################################################################################
    in_ch=x.get_shape()[3]
    out_ch1=192 ; out_ch2=192;

    c_ksize1 =[1,1,in_ch , out_ch1]
    c_ksize2 =[3,3,out_ch1,out_ch2]
    w_conv1, b_conv1 = make_weights_biases(layer_name+'WB1','W1','B1',c_ksize1,device)
    w_conv2, b_conv2 = make_weights_biases(layer_name+'WB2','W2','B2',c_ksize2,device)
    
    c_strides1=[1,1,1,1]
    c_strides2=[1,2,2,1]
    
    c_pooling1='SAME'
    c_pooling2='VALID'
    with tf.variable_scope(layer_name+'layer1') as scope:
        layer1 = tf.nn.conv2d(x,w_conv1 , c_strides1, c_pooling1 , name='conv2d')
        layer1_relu=tf.nn.relu(layer1,name='relu')
    with tf.variable_scope(layer_name+'layer2') as scope:
        layer2 = tf.nn.conv2d(layer1_relu , w_conv2 , c_strides2 , c_pooling2 , name='conv2d')    
        layer2_relu=tf.nn.relu(layer2 , name ='relu')
    ####################################################################################
    b1_p_ksize=[1,3,3,1]
    b1_p_strides=[1,2,2,1]
    b1_p_pooling='VALID'
    with tf.variable_scope(layer_name+'b1_layer') as scope:
        b1_layer=tf.nn.max_pool(x ,b1_p_ksize , b1_p_strides , b1_p_pooling,name='max_pool')
        b1_layer_relu = tf.nn.relu(b1_layer , name='relu')
    ####################################################################################
    b2_out_ch1=256 ; b2_out_ch2 = 256 ; b2_out_ch3=320 ; b2_out_ch4=320;
    b2_c_ksize1 =[1,1,in_ch , b2_out_ch1]
    b2_c_ksize2 =[1,7,b2_out_ch1 , b2_out_ch2]
    b2_c_ksize3 =[7,1,b2_out_ch2 , b2_out_ch3]
    b2_c_ksize4 =[3,3,b2_out_ch3 , b2_out_ch4]
    
    b2_w_conv1, b2_b_conv1 = make_weights_biases(layer_name+'b2_WB_1','b2_W1' ,'b2_B1',b2_c_ksize1,device)
    b2_w_conv2, b2_b_conv2 = make_weights_biases(layer_name+'b2_WB_2','b2_W2' ,'b2_B2',b2_c_ksize2,device)
    b2_w_conv3, b2_b_conv3 = make_weights_biases(layer_name+'b2_WB_3','b2_W3' ,'b2_B3',b2_c_ksize3,device)
    b2_w_conv4, b2_b_conv4 = make_weights_biases(layer_name+'b2_WB_4','b2_W4' ,'b2_B4',b2_c_ksize4,device)
    
    b2_c_strides1=[1,1,1,1]
    b2_c_strides2=[1,1,1,1]
    b2_c_strides3=[1,1,1,1]
    b2_c_strides4=[1,2,2,1]
    
    b2_c_pooling1= 'SAME'
    b2_c_pooling2= 'SAME'
    b2_c_pooling3 = 'SAME'
    b2_c_pooling4 = 'VALID'
    with tf.variable_scope(layer_name+'b2_layer1') as scope:
        b2_layer1 = tf.nn.conv2d(x, b2_w_conv1 , b2_c_strides1 , b2_c_pooling1 ,name='conv2d')+b2_b_conv1
        b2_layer1_relu = tf.nn.relu(b2_layer1 , name='relu') 
    with tf.variable_scope(layer_name+'b2_layer1') as scope:
        b2_layer2 = tf.nn.conv2d(b2_layer1_relu, b2_w_conv2 , b2_c_strides2 , b2_c_pooling2,name='conv2d' )+b2_b_conv2
        b2_layer2_relu = tf.nn.relu(b2_layer2 , name='relu')
    with tf.variable_scope(layer_name+'b2_layer2') as scope:
        b2_layer3 = tf.nn.conv2d(b2_layer2_relu, b2_w_conv3 , b2_c_strides3 , b2_c_pooling3 ,name='conv2d' )+b2_b_conv3
        b2_layer3_relu = tf.nn.relu(b2_layer3 , name='relu')
    with tf.variable_scope(layer_name+'b2_layer1') as scope:
        b2_layer4 = tf.nn.conv2d(b2_layer3_relu, b2_w_conv4 , b2_c_strides4 , b2_c_pooling4 ,name='conv2d')+b2_b_conv4
        b2_layer4_relu=tf.nn.relu(b2_layer4 , name='relu')
    with tf.variable_scope(layer_name+'concat') as scope:
        concat_A=tf.concat(3,[layer2_relu,b1_layer_relu] ,name='concat')
        concat_B=tf.concat(3,[concat_A,b2_layer4_relu] ,name='concat')
        
    return concat_B
    ####################################################################################

# FULLYCONNCECT

In [17]:
def FLAT(x):
    
    row=int(x.get_shape()[1])
    col=int(x.get_shape()[2])
    ch=int(x.get_shape()[3])
    
    res_x = tf.reshape(x , shape=[-1,row*col*ch] ,name='flat_layer');
    return res_x
    #connect fully connected layer 

In [18]:
def FC_A(x , n_classes , device_ ,dropout_rate):
    with tf.device(device_):
    
        fully_ch1=1024; fully_ch2=1024

        fc_ksize1=[x.get_shape()[1],fully_ch1]
        fc_ksize2=[fully_ch1,fully_ch2]

        w_fc1 ,b_fc1 = make_weights_biases('fc1' , 'fc_W1' ,'fc_B1' ,fc_ksize1 ,  device_)
        w_fc2 ,b_fc2 = make_weights_biases('fc2' , 'fc_W2' ,'fc_B2' ,fc_ksize2 ,  device_)
    
        h_fc1=tf.matmul(x, w_fc1 ,name='h_fc1')+b_fc1
        h_fc1=tf.nn.dropout(h_fc1 , dropout_rate , name='h_fc1_dropout')
        h_fc2=tf.matmul(h_fc1 , w_fc2 ,name='h_fc2')+b_fc2
        h_fc2=tf.nn.dropout(h_fc2 , dropout_rate,name='h_fc2_dropout')
        end_fc=h_fc2

        end_ksize=[end_fc.get_shape()[1] , n_classes]   
        w_end ,b_end = make_weights_biases('fc_end' , 'fc_end_W' , 'fc_end_B' ,end_ksize ,  device_)
        y_conv = tf.matmul(end_fc , w_end)+b_end

        print w_fc1.get_shape()
    return y_conv
    

In [19]:
def FC_B(x , n_classes , device_ ):
    with tf.device(device_):

        end_ksize=[x.get_shape()[1] , n_classes]   
        w_end ,b_end = make_weights_biases('fc_end' , 'fc_end_W','fc_end_B' , end_ksize ,  device_)
        y_conv = tf.matmul(x , w_end)+b_end

   
    return y_conv

In [20]:
def AVG_POOL(x , ksize , strides ,pooling,layer_name,device_):
    with tf.device(device_):
        layer=tf.nn.avg_pool(x,ksize ,strides,pooling,name = layer_name+"AVG_POOL")
    return layer

In [21]:
def MAX_POOL(x , ksize , strides,pooling,layer_name,device_):
    with tf.device(device_):
        layer=tf.nn.avg_pool(x,ksize ,strides,pooling,name = layer_name+"MAX_POOL")
    return layer
    

In [22]:
def DROPOUT(x ,keep_prob , layer_name , device_):
    with tf.device(device_):
        drop_x=tf.nn.dropout(x , keep_prob , name=layer_name+'dropout')
        return drop_x

# CONVOLUTION LAYER

In [23]:
def CONV_6(x , device_ , layer_name , n_nodes , kernel_size ,strides ,pooling):
    """ 
    k_row=kernel_size[0]  
    k_col=kernel_size[0] 
    
    """
    in_ch = x.get_shape()[3]
    k_row=kernel_size[0]  
    k_col=kernel_size[1] 
    
    
    out_ch1=n_nodes
    out_ch2=n_nodes
    out_ch3=n_nodes
    out_ch4=n_nodes
    out_ch5=n_nodes
    out_ch6=n_nodes
    
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    c_ksize4=[k_row,k_col , out_ch3 , out_ch4]
    c_ksize5=[k_row,k_col , out_ch4 , out_ch5]
    c_ksize6=[k_row,k_col , out_ch5 , out_ch6]
    
    c_strides1=strides
    c_strides2=strides
    c_strides3=strides
    c_strides4=strides
    c_strides5=strides
    c_strides6=strides
    
    c_pooling1=pooling
    c_pooling2=pooling
    c_pooling3=pooling
    c_pooling4=pooling
    c_pooling5=pooling
    c_pooling6=pooling
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1) , 'W1' ,'B1', c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2) , 'W2' ,'B2',c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3) , 'W3' ,'B3' ,c_ksize3 ,device_name = device_)
    w_conv4 , b_conv4= make_weights_biases(layer_name+str(4) , 'W4' ,'B4', c_ksize4 ,device_name = device_)
    w_conv5 , b_conv5= make_weights_biases(layer_name+str(5) , 'W5' ,'B5', c_ksize5 ,device_name = device_)
    w_conv6 , b_conv6= make_weights_biases(layer_name+str(6) , 'W6' ,'B6', c_ksize5 ,device_name = device_)

    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3,name='layer3')+b_conv3
        layer3 = tf.nn.relu(layer3,name='layer3_relu')
    with tf.variable_scope(layer_name+'conv4') as scope:
        layer4 = tf.nn.conv2d(layer3 , w_conv4 , c_strides4 , c_pooling4,name='layer4')+b_conv4
        layer4 = tf.nn.relu(layer4,name='layer4_relu')
    with tf.variable_scope(layer_name+'conv5') as scope:
        layer5 = tf.nn.conv2d(layer4 , w_conv5 , c_strides5 , c_pooling5,name='layer5')+b_conv5
        layer5 = tf.nn.relu(layer5,name='layer5_relu')
    with tf.variable_scope(layer_name+'conv6') as scope:
        layer6 = tf.nn.conv2d(layer5 , w_conv6 , c_strides6 , c_pooling6,name='layer6')+b_conv6
        layer6 = tf.nn.relu(layer5,name='layer6_relu')

    return layer6

In [24]:
def CONV_5(x , device_ , layer_name , n_nodes , kernel_size ,strides ,pooling):
    """ 
    k_row=kernel_size[0]  
    k_col=kernel_size[0] 
    
    """
    in_ch = x.get_shape()[3]
    k_row=kernel_size[0]  
    k_col=kernel_size[1] 
    
    
    out_ch1=n_nodes
    out_ch2=n_nodes
    out_ch3=n_nodes
    out_ch4=n_nodes
    out_ch5=n_nodes
    
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    c_ksize4=[k_row,k_col , out_ch3 , out_ch4]
    c_ksize5=[k_row,k_col , out_ch4 , out_ch5]
    
    c_strides1=strides
    c_strides2=strides
    c_strides3=strides
    c_strides4=strides
    c_strides5=strides
    
    c_pooling1=pooling
    c_pooling2=pooling
    c_pooling3=pooling
    c_pooling4=pooling
    c_pooling5=pooling
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1), 'W1' ,'B1' , c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2), 'W2' ,'B2' , c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3), 'W3' ,'B3' , c_ksize3 ,device_name = device_)
    w_conv4 , b_conv4= make_weights_biases(layer_name+str(4), 'W4' ,'B4' , c_ksize4 ,device_name = device_)
    w_conv5 , b_conv5= make_weights_biases(layer_name+str(5), 'W5' ,'B5' , c_ksize5 ,device_name = device_)

    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3,name='layer3')+b_conv3
        layer3 = tf.nn.relu(layer3,name='layer3_relu')
    with tf.variable_scope(layer_name+'conv4') as scope:
        layer4 = tf.nn.conv2d(layer3 , w_conv4 , c_strides4 , c_pooling4,name='layer4')+b_conv4
        layer4 = tf.nn.relu(layer4,name='layer4_relu')
    with tf.variable_scope(layer_name+'conv5') as scope:
        layer5 = tf.nn.conv2d(layer4 , w_conv5 , c_strides5 , c_pooling5,name='layer5')+b_conv5
        layer5 = tf.nn.relu(layer5,name='layer4_relu')

    return layer5

In [25]:
def CONV_3(x , device_ , layer_name , n_nodes , kernel_size ,strides,pooling):
    """
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    """
    
    in_ch = x.get_shape()[3]
    k_row=kernel_size[0]  
    k_col=kernel_size[1]     
    
    c_ksize1=[k_row,k_col , in_ch   , n_nodes]
    c_ksize2=[k_row,k_col , n_nodes , n_nodes]
    c_ksize3=[k_row,k_col , n_nodes , n_nodes]
    
    c_strides1=strides
    c_strides2=strides
    c_strides3=strides
        
    c_pooling1=pooling
    c_pooling2=pooling
    c_pooling3=pooling
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1) , 'W1' ,'B1', c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2) , 'W2' ,'B2',c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3) , 'W3' ,'B3' ,c_ksize3 ,device_name = device_)
    
    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3,name='layer3')+b_conv3
        layer3 = tf.nn.relu(layer3,name='layer3_relu')
    return layer3

In [26]:
def CONV_1(x , device_ , layer_name , n_nodes , kernel_size ,strides,pooling):
    """
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    """
    
    in_ch = x.get_shape()[3]
    k_row=kernel_size[0]  
    k_col=kernel_size[1]     
    
    c_ksize1=[k_row,k_col , in_ch   , n_nodes]
    c_strides1=strides        
    c_pooling1=pooling
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1) , 'W1' ,'B1', c_ksize1 ,device_name = device_)
    
    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    return layer1

In [27]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]




In [28]:
def get_batch_list(folder_path):
    """
    return ret_train_img_list ,ret_train_lab_list 
    
    """
    list_files=os.walk(folder_path).next()[2]
    ret_train_img_list=[]
    ret_train_lab_list=[]
    np_train_imgs=[]
    np_train_labs=[]
    for i , ele in enumerate(list_files):

        if 'train'  in ele and 'img'in ele:
            ret_train_img_list.append(ele)
        elif 'train' in ele  and  'lab' in ele:
            ret_train_lab_list.append(ele)
    
    ret_train_img_list.sort(key=natural_keys)
    ret_train_lab_list.sort(key = natural_keys)
    for i  in range(len(ret_train_img_list)):
        np_train_imgs.append(np.load(folder_path+ret_train_img_list[i]))
        np_train_labs.append(np.load(folder_path+ret_train_lab_list[i]))
    
    return np_train_imgs ,np_train_labs 

In [29]:
def aug(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col , label):
    
    n_img = np.shape(np_img)[0] 
    n_ret_img = n_img*(img_row - crop_img_row) * (img_col - crop_img_col)*2
    ret_images = np.zeros([n_ret_img ,crop_img_row , crop_img_col,color_ch])
    len_label= np.shape(label)[1]
    ret_labels =  np.zeros([n_ret_img  ,len_label])
    #print "n_augmented image size : " , n_ret_img 
    #print "n classes :", len_label
    #copy label in factor by 2014
    
    if len(np.shape(np_img))==2:
        np_img=np.reshape(np_img , newshape = [np.shape(np_img)[0] , img_row , img_col ,color_ch])
        print np.shape(np_img)
    for n  in range(len(np_img)):
        ret_labels[n*2 , : ] = label[n,:]
        ret_labels[n*2+1 , : ] = label[n,:]
        for r in range(img_row - crop_img_row):
            for c in range(img_col - crop_img_col):
                
                cropped_img = np_img[n, r:crop_img_row +r , c:crop_img_col+c ,: ]
                
                ret_images[n*2,:,:,:]=cropped_img  
                ret_images[(n*2+1) , :,:,:] =np.fliplr(cropped_img )

    
    return ret_images ,ret_labels



In [30]:
def extract_test_img(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col ):
    left_top =(0,0)
    right_top =(  img_row  - crop_img_row  , 0 )
    center =  ((img_row  - crop_img_row)/2  , (img_col - crop_img_row)/2)
    left_buttom = (0,(img_col - crop_img_row)/2 )
    right_buttom =  (img_row  - crop_img_row , img_col - crop_img_row)
    
    left_top_images  = np_img[: , left_top[0]:crop_img_row+left_top[0] , left_top[1] : crop_img_col+left_top[1] , :  ]
    right_top_images = np_img[: , right_top[0]:crop_img_row +right_top[0], right_top[1] : crop_img_col +right_top[1], :  ]
    center_images    = np_img[: , center[0]:crop_img_row +center[0], center[1] : crop_img_col +center[1], :  ]
    left_buttom_images=np_img[: , left_buttom[0]:crop_img_row +left_buttom[0], left_buttom[1] : crop_img_col +left_buttom[1], :  ]
    right_buttom_images= np_img[: , right_buttom[0]:crop_img_row+right_buttom[0] , right_buttom[1] : crop_img_col +right_buttom[1] , :  ]

    
        
    return left_top_images , right_top_images , center_images , left_buttom_images , right_buttom_images 



In [31]:
def TRAIN_STRUCTURE_A(y_conv , y_ , device_,learning_rate=1e-4 ):
    """
    Return Value : cost , train_step ,correct_prediction , accuracy 
    
    """
    with tf.device(device_):
    #sm_conv= tf.nn.softmax(y_conv)
        #cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

        softmax=tf.nn.softmax(y_conv)
        regular=0.01*(tf.reduce_sum(tf.square(y_conv)))
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( y_conv, y_))
    with tf.device(device_):
        cost = cost+regular
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(cost) #1e-4
        with tf.name_scope("accuracy"):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.argmax(y_conv,1) ,tf.argmax(y_,1))
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float")) 

    return cost , train_step ,correct_prediction , accuracy ,softmax



In [32]:
def START_SESS():
    sess = tf.Session()
  
    sess.run(tf.initialize_all_variables())
    return sess 


# File Terminal Open

In [33]:
#dirname = '/home/ncc/notebook/mammo/result/'
def make_logdir(dirname):
  

    count=0
    while(True):
        if not os.path.isdir(dirname):
            os.mkdir(dirname)
            break
        elif not os.path.isdir(dirname + str(count)):
            dirname=dirname+str(count)
            os.mkdir(dirname)
            break
        else:
            count+=1
    print 'it is recorded at :'+str(count)

    f=open(dirname+"/log.txt",'w')
    return f

In [34]:

def write_log(step,train_acc, train_loss , val_acc , val_loss ,fp):
    """
    fp = File Pointer
    
    """
    #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
    print step
    print("step %d , training  accuracy %g" %(step,train_acc))
    print("step %d , loss : %g" %(step,train_loss))
    train_str = 'step:\t'+str(step)+'\ttrain_loss:\t'+str(train_loss) +'\ttrain_accuracy:\t'+str(train_acc)+'\n'

    print("step %d , validation  accuracy %g" %(step,val_acc))
    print("step %d , validation loss : %g" %(step,val_loss))
    val_str = 'step:\t'+str(step)+'\tval_loss:\t'+str(val_loss) +'\tval_accuracy:\t'+str(val_acc)+'\n'

    fp.write(train_str)
    fp.write(val_str)
    

In [35]:
def load_TVT(divide_flag,file_locate):
    if divide_flag == False:
        train_img=np.load(file_locate+'train_img.npy');
        train_lab=np.load(file_locate+'train_lab.npy');
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');

        print "Training Data Image",np.shape(train_img)
        print "Training Data Label",np.shape(train_lab)
        print "Test Data Label",np.shape(test_lab)
        print "Test Data Image",np.shape(test_img)
        print "val Data Label" , np.shape(val_lab)
        print "val Data Image" , np.shape(val_img)
        
        n_train= np.shape(train_img)[0]
        n_train_lab = np.shape(train_lab)[0]
        return train_img ,train_lab,val_img,val_lab,test_img,test_lab
    if divide_flag == True:
        print '트레이닝 파일이 여러개로 분할되어 있습니다. 분할된 트레이닝 파일에 대한 조치가 필요합니다'
        train_images, train_labels =get_batch_list(file_locate)
        #print train_images ,train_labels
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');
        print "the number of training image batch",len(train_images)
        print "the number of training label batch",len(train_labels)
        print "training Data Label",np.shape(train_labels[0])
        print "training Data Image",np.shape(train_images[0])
        print "Test Data Label",np.shape(test_lab)
        print "Test Data Image",np.shape(test_img)
        print "val Data Label" , np.shape(val_lab)
        print "val Data Image" , np.shape(val_img)
        return train_images, train_labels,val_img,val_lab,test_img,test_lab

In [36]:
def validate_from_images(sess, images , labels , tensor_info, place_info , tensorboard_info = None):    
    """
    input : x-
    
    x type  : numpy 
    x shape : [n , row , col , ch]
    return  acc,  loss

    """
    
    acc_list=[]
    loss_list=[]
    images_labels=zip(images,labels)
    for img_ind ,(img,lab)  in enumerate(images_labels):

        acc ,loss = sess.run( [tensor_info['accuracy'],tensor_info['cost']] ,\
                             feed_dict={place_info['x_']:img , place_info['y_']: lab , place_info['keep_prob']: 1.0})        
        acc_list.append(acc)
        loss_list.append(loss)

    acc_list=np.asarray(acc_list)
    loss_list=np.asarray(loss_list)
    acc=np.mean(acc_list)
    loss=np.mean(loss_list)
    
    
    
    return  acc,  loss


In [37]:
def validate(img , lab ,tensor_info , place_info,tensorboard_info=None , step=None): #default
    """
    return val_acc,  val_loss, train_acc, train_loss
    tensorboard_info['writer']
    tensorboard_info['merge']    
    """    
    print 'validate tensorboard' , tensorboard_info
    if tensorboard_info ==None:
        print 'b'
        acc,loss = sess.run([tensor_info['accuracy'],tensor_info['cost']] , feed_dict={place_info['x_']:img , place_info['y_']:lab , place_info['keep_prob']: 1.0})        
    else:
        print 'a'
        summary,acc,loss = sess.run([tensorboard_info['merge'],tensor_info['accuracy'],tensor_info['cost']] , feed_dict={place_info['x_']:img , place_info['y_']:lab , place_info['keep_prob']: 1.0})        
        tensorboard_info['writer'].add_summary(summary , step)
        
        
    return acc,loss

In [38]:
def BATCH_TRAINING_DIVIDE_RANDOM(maxiter, batch_size,file_locate,tensor_info, place_info , fp):
    
    
    start_time = time.time()
    #with tf.device('/gpu:0'):
    train_imgs, train_labs,val_img,val_lab,test_img,test_lab=load_TVT(True,file_locate) #TVT is Train Validation Test  
    for step in range(maxiter):
        b_ind=random.randrange(0,len(train_imgs))
        train_img = train_imgs[b_ind]
        train_lab = train_labs[b_ind]
        
        batch_xs , batch_ys = next_batch(batch_size, train_img , train_lab)       
        if step%100 ==0: #여기 if loop에서 validate을 합니다 
            try:
                val_acc , val_loss= validate(val_img,val_lab,tensor_info,place_info)     
                train_acc , train_loss=validate(batch_xs,batch_ys,tensor_info,place_info)
            except:
                #한번에 트레이닝이 안되면 분할해서 validation한다.
                val_imgs, val_labs = divide_images(val_img , val_lab, batch_size)
                #print np.shape(val_imgs[0])
                val_acc, val_loss =validate_from_images(sess,val_imgs ,val_labs ,tensor_info ,place_info )
                train_acc , train_loss=validate(batch_xs,batch_ys,tensor_info ,place_info)
                
            write_log(step , train_acc, train_loss , val_acc, val_loss , fp)
            #softmax_=sess.run(softmax ,feed_dict ={ x:batch_xs , y_:batch_ys , keep_prob:1.0 })
            #print softmax_
        training(batch_xs,batch_ys,tensor_info,place_info)

    ############################################aug_and_training############################
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    
    try:
        test_acc , test_loss=validate(test_img , test_lab)
    except:
        #한번에 트레이닝이 안되면 분할해서 validation한다.
        test_imgs, test_labs = divide_images(test_img ,test_lab, batch_size)
        #print np.shape(val_imgs[0])
        test_acc, test_loss =validate_from_images(sess,test_imgs ,test_labs ,tensor_info,place_info )
    print "test accuracy {0} , test loss {1}".format(test_acc, test_loss)
    fp.write(train_time)
    fp.close()
    sess.close()

In [39]:
def BATCH_TRAINING_RANDOM(maxiter, batch_size, file_locate, tensor_info , place_info, fp , tensorboard_info=None):
    print tensorboard_info
    start_time = time.time()
    #with tf.device('/gpu:0'):
    train_img, train_lab,val_img,val_lab,test_img,test_lab=load_TVT(False,file_locate) #TVT is Train Validation Test  
    for step in range(maxiter):    
        batch_xs , batch_ys = next_batch(batch_size, train_img , train_lab)       
        if step%100 ==0: #여기 if loop에서 validate을 합니다 
            try:
                if tensorboard_info == None:
                    print 'none'
                    val_acc , val_loss = validate(val_img , val_lab,tensor_info,place_info,tensorboard_info ,step)
                    train_acc , train_loss = validate(batch_xs,batch_ys,tensor_info,place_info)
                
                else: 
                    val_acc , val_loss = validate(val_img , val_lab ,tensor_info,place_info,tensorboard_info ,step)
                    train_acc , train_loss = validate(batch_xs,batch_ys )
                
            except:
                #한번에 트레이닝이 안되면 분할해서 validation한다.
                val_imgs, val_labs = divide_images(val_img , val_lab, batch_size)
                #print np.shape(val_imgs[0])
                val_acc, val_loss =validate_from_images(sess, val_imgs ,val_labs ,tensor_info ,place_info,tensorboard_info)
                train_acc , train_loss=validate(batch_xs,batch_ys,tensor_info ,place_info)
                       
            write_log(step , train_acc, train_loss , val_acc, val_loss , fp)
            #softmax_=sess.run(softmax ,feed_dict ={ x:batch_xs , y_:batch_ys , keep_prob:1.0 })
            #print softmax_
        training(batch_xs,batch_ys,tensor_info , place_info)

    ############################################aug_and_training############################
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    
    try:
        test_acc , test_loss=validate(test_img , test_lab)
    except:
        #한번에 트레이닝이 안되면 분할해서 validation한다.
        test_imgs, test_labs = divide_images(test_img ,test_lab, batch_size)
        #print np.shape(val_imgs[0])
        test_acc, test_loss =validate_from_images(sess,test_imgs ,test_labs ,tensor_info,place_info )
    print "test accuracy {0} , test loss {1}".format(test_acc, test_loss)
    fp.write(train_time)
    fp.close()

In [40]:
def BATCH_TRAINING_MANUAL(maxiter  , batch_size,file_locate, tensor_info, place_info , fp ):
    start_time = time.time()
#with tf.device('/gpu:0'):
    train_img, train_lab,val_img,val_lab,test_img,test_lab\
    =load_TVT(divide_flag,file_locate)
    train_images , train_labels=divide_images(train_img , train_lab , batch_size)
    train_images_labels=zip(train_images , train_labels)
    
    for step,(batch_xs , batch_ys) in train_images_labels:
        if step%100 ==0:
            try:
                val_acc , val_loss=validate(val_img , val_lab)
                train_acc , train_loss=validate(batch_xs,batch_ys)
            except:
                #한번에 트레이닝이 안되면 분할해서 validation한다.
                val_imgs, val_labs = divide_images(val_img , val_lab, batch_size)
                val_acc, val_loss =validate_from_images(val_imgs ,val_labs ,accuracy, cost )
                train_acc , train_loss=validate(batch_xs,batch_ys,accuracy, cost)
            write_log(step , train_acc, train_loss , val_acc, val_loss , fp)
            np.save('./result/batch_xs' ,batch_xs)
            np.save('./result/batch_ys' ,batch_ys)
        else:
            list_aud_x = aug_8_times(batch_xs)
            for ele in list_aug_x:
                training(ele ,batch_ys)                    


############################################aug_and_training############################
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    fp.write(train_time)
    fp.close()

In [41]:

def divide_images(images , labels,batch_size):
    """
    return list_images,list_labels
    """
    n_divide=len(images)/batch_size

    list_images=[]
    list_labels=[]
    for ind in range(n_divide):
        # j*batch_size :(j+1)*batch_size
        image =images[ ind*batch_size :(ind+1)*batch_size] 
        label =labels[ ind*batch_size :(ind+1)*batch_size]
        list_images.append(image)
        list_labels.append(label)

    #right above code have to modify
    image = images[ (ind+1)*batch_size :  ] 
    label = labels[ (ind+1)*batch_size :  ]
    list_images.append(image)
    list_labels.append(label)

    return list_images,list_labels

In [42]:
  
def validate_extract_imgs(val_img , val_lab , train_img , train_lab ):
    """
    extract patch from ori-image
    """
    color_ch = in_ch
    val_images  =extract_test_img(val_img , 128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
    train_images=extract_test_img(train_img ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
    val_acc, val_loss =validate_from_images(sess, val_images , val_lab , accuracy ,cost )
    train_acc , train_loss =validate_from_images(sess, train_images , train_lab ,accuracy , cost)
    
    return val_acc , val_loss, train_acc ,train_loss
    


In [43]:

def training(batch_xs , batch_ys , tensor_info , place_info):
    sess.run(tensor_info['train_step'] ,feed_dict={place_info['x_']: batch_xs, place_info['y_']:batch_ys , place_info['keep_prob'] : 0.5})

In [44]:
def aug_8_times(x):
    """
    
    
    x shape is [n_batch , row ,col , color_ch ]
    x type is numpy 
    this code need too many time to run 
    we should find solution using parallel method to less run time maybe 
    
    return x,np_rot90,np_rot180,np_rot270,lr_x,np_lr_rot90 ,np_lr_rot180 , np_lr_rot270 
    
    """

    n_batch,row,col,ch=np.shape(x)
    lr_x = np.flipud(x)

    np_rot90 =np.zeros(shape =[n_batch , row ,col ,ch] )
    np_rot180=np.zeros(shape =[n_batch , row ,col ,ch] )
    np_rot270=np.zeros(shape =[n_batch , row ,col ,ch] )

    np_lr_rot90 =np.zeros(shape =[n_batch , row ,col ,ch] )
    np_lr_rot180=np.zeros(shape =[n_batch , row ,col ,ch] )
    np_lr_rot270=np.zeros(shape =[n_batch , row ,col ,ch] )
    
    

    for batch_ind in range(n_batch):

        rot90=np.rot90(x[batch_ind,:,:,:])
        rot180=np.rot90(rot90)
        rot270=np.rot90(rot180)

        np_rot90[batch_ind,:,:,:] = rot90
        np_rot180[batch_ind,:,:,:]=rot180
        np_rot270[batch_ind,:,:,:]=rot270

        lr_rot90=np.rot90(lr_x[batch_ind,:,:,:])
        lr_rot180=np.rot90(lr_rot90)
        lr_rot270=np.rot90(lr_rot180)

        np_lr_rot90[batch_ind,:,:,:]=lr_rot90
        np_lr_rot180[batch_ind,:,:,:]=lr_rot180
        np_lr_rot90[batch_ind,:,:,:]=lr_rot270
    return x,np_rot90,np_rot180,np_rot270,lr_x,np_lr_rot90 ,np_lr_rot180 , np_lr_rot270 
    

In [45]:
def OPEN_TENSORBOARD(sess,tensor_info , logdir ):
    
    tensorboard_info={}
    tb_acc=tf.summary.scalar("accuarcy" ,tensor_info['accuracy'] )
    tb_loss=tf.summary.scalar("loss" ,tensor_info['cost'] )
    tb_merge = tf.summary.merge_all()
    writer=tf.train.SummaryWriter(logdir , sess.graph )
    sess.run(tf.global_variables_initializer())
    
    
    tensorboard_info['writer']=writer
    tensorboard_info['merge'] =tb_merge
  
    return tensorboard_info

In [ ]:
def aug_crop(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col , label):
    n_img = np.shape(np_img)[0] 
    n_ret_img = n_img*(img_row - crop_img_row) * (img_col - crop_img_col)*2
    ret_images = np.zeros([n_ret_img ,crop_img_row , crop_img_col,color_ch])
    len_label= np.shape(label)[1]
    ret_labels =  np.zeros([n_ret_img  ,len_label])
    #print "n_augmented image size : " , n_ret_img 
    #print "n classes :", len_label
    #copy label in factor by 2014

    if len(np.shape(np_img))==2:
        np_img=np.reshape(np_img , newshape = [np.shape(np_img)[0] , img_row , img_col ,color_ch])
        print np.shape(np_img)
    for n  in range(len(np_img)):
        ret_labels[n*2 , : ] = label[n,:]
        ret_labels[n*2+1 , : ] = label[n,:]
        for r in range(img_row - crop_img_row):
            for c in range(img_col - crop_img_col):

                cropped_img = np_img[n, r:crop_img_row +r , c:crop_img_col+c ,: ]

                ret_images[n*2,:,:,:]=cropped_img  
                ret_images[(n*2+1) , :,:,:] =np.fliplr(cropped_img )


    return ret_images ,ret_labels



# TEST

In [ ]:
tensor_info={};place_info={}
file_locate='./eye_numpy_64/'
dirname='./result_eye/'
"""
A=CONV_1(x_ , '/gpu:1' , 'CONVx5_1',16,(3,3),[1,2,2,1],'SAME')
B=CONV_5(A  , '/gpu:1' , 'CONVx5_2',16,(3,3),[1,1,1,1],'SAME')
C=CONV_5(B  , '/gpu:1' , 'CONVx5_3',16,(3,3),[1,1,1,1],'SAME')
D=CONV_5(C  , '/gpu:1' , 'CONVx5_4',16,(3,3),[1,1,1,1],'SAME')

E=CONV_1(D  , '/gpu:1' , 'CONVx5_5',32,(3,3),[1,2,2,1],'SAME')
F=CONV_5(E  , '/gpu:1' , 'CONVx5_6',32,(3,3),[1,1,1,1],'SAME')
G=CONV_5(F  , '/gpu:1' , 'CONVx5_7',32,(3,3),[1,1,1,1],'SAME')
H=CONV_5(G  , '/gpu:1' , 'CONVx5_8',32,(3,3),[1,1,1,1],'SAME')

I=CONV_1(H  , '/gpu:1' , 'CONVx5_9',64,(3,3),[1,2,2,1],'SAME')
J=CONV_5(I  , '/gpu:1' , 'CONVx5_10',64,(3,3),[1,1,1,1],'SAME')
K=CONV_5(J  , '/gpu:1' , 'CONVx5_11',64,(3,3),[1,1,1,1],'SAME')
L=CONV_5(K  , '/gpu:1' , 'CONVx5_12',64,(3,3),[1,1,1,1],'SAME')
"""
A=STEM_A(x_ ,'/gpu:4')
B=STEM_B(A , '/gpu:4')
C=STEM_C(B , '/gpu:4')

D=INCEPTION_MODULE_A(C,'/gpu:1', 'INCEPTION_MODULE_A_1')
E=INCEPTION_MODULE_A(D,'/gpu:1', 'INCEPTION_MODULE_A_2')
F=INCEPTION_MODULE_A(E,'/gpu:1', 'INCEPTION_MODULE_A_3')
G=INCEPTION_MODULE_A(F,'/gpu:1', 'INCEPTION_MODULE_A_4')

H=INCEPTION_REDUCTION_A(G,'/gpu:2' , 'INCEPTION_REDUCTION_A_1')

I=INCEPTION_MODULE_B(H,'/gpu:2' , 'INCEPTION_MODULE_B_1')
J=INCEPTION_MODULE_B(I,'/gpu:2' , 'INCEPTION_MODULE_B_2')
K=INCEPTION_MODULE_B(J,'/gpu:2' , 'INCEPTION_MODULE_B_3')
L=INCEPTION_MODULE_B(K,'/gpu:2' , 'INCEPTION_MODULE_B_4')
M=INCEPTION_MODULE_B(L,'/gpu:2' , 'INCEPTION_MODULE_B_5')
N=INCEPTION_MODULE_B(M,'/gpu:3' , 'INCEPTION_MODULE_B_6')
O=INCEPTION_MODULE_B(N,'/gpu:3' , 'INCEPTION_MODULE_B_7')

P=INCEPTION_REDUCTION_B(O , '/gpu:3' , 'INCEPTION_REDUCTION_B_1')
Q=INCEPTION_MODULE_C(P,'/gpu:3' , 'INCEPTION_MODULE_C_1')
R=INCEPTION_MODULE_C(Q,'/gpu:3' , 'INCEPTION_MODULE_C_2')
S=INCEPTION_MODULE_C(R,'/gpu:3' , 'INCEPTION_MODULE_C_3')
T=AVG_POOL(S,[1,2,2,1],[1,1,1,1],'SAME','AVG_POOL','/gpu:3 ')
U=DROPOUT(T , 0.8 ,'INCEPTION_END','/gpu:3')

flat_=FLAT(U)
y_conv=FC_B(flat_ , n_classes ,'/gpu:3' )
#y_conv=FC_A(flat_ , n_classes ,'/gpu:1' ,dropout_rate=keep_prob)
cost , train_step ,correct_prediction , accuracy ,softmax = \
TRAIN_STRUCTURE_A(y_conv , y_ , '/gpu:3' , learning_rate = 0.0001)

tensor_info['cost']=cost
tensor_info['train_step']=train_step
tensor_info['correct_prediction']=correct_prediction
tensor_info['accuracy']=accuracy
tensor_info['softmax']=softmax

place_info['x_']=x_
place_info['y_']=y_
place_info['keep_prob']=keep_prob


logdir='./eye_128/INCEPTION_v4/2'
sess=START_SESS()
tensorboard_info=OPEN_TENSORBOARD(sess ,tensor_info ,logdir)
fp=make_logdir(dirname)
BATCH_TRAINING_RANDOM(100000 , 30,file_locate , tensor_info,place_info ,fp ,tensorboard_info)
#BATCH_TRAINING_DIVIDE_RANDOM(100000 , 30,file_locate , tensor_info,place_info ,fp)


Tensor("STEM_A_layer1/add:0", shape=(?, 63, 63, 32), dtype=float32, device=/device:GPU:4)
Tensor("STEM_A_layer2/add:0", shape=(?, 61, 61, 32), dtype=float32, device=/device:GPU:4)
Tensor("STEM_A_layer3/add:0", shape=(?, 61, 61, 64), dtype=float32, device=/device:GPU:4)
Tensor("STEM_A_layer4/add:0", shape=(?, 30, 30, 96), dtype=float32, device=/device:GPU:4)
Tensor("STEM_A_b_layer/max_pool:0", shape=(?, 30, 30, 64), dtype=float32, device=/device:GPU:4)
Tensor("STEM_C_layer/add:0", shape=(?, 28, 28, 192), dtype=float32, device=/device:GPU:4)
Tensor("STEM_C_b_layer/max_pool:0", shape=(?, 28, 28, 192), dtype=float32, device=/device:GPU:4)
(?, 28, 28, 384)
(?, 28, 28, 384)
(?, 28, 28, 384)
(?, 28, 28, 384)
(?, 28, 28, 384)
(?, 13, 13, 384)
(?, 13, 13, 384)
(?, 13, 13, 256)
Tensor("INCEPTION_MODULE_B_1concat/3:0", shape=(?, 13, 13, 1024), dtype=float32)
Tensor("INCEPTION_MODULE_B_2concat/3:0", shape=(?, 13, 13, 1024), dtype=float32)
Tensor("INCEPTION_MODULE_B_3concat/3:0", shape=(?, 13, 13, 

In [ ]:
tensor_info={};place_info={}
file_locate='./eye_numpy_64/'
dirname='./result_eye/'

A=PRE_LAYER(x_ , '/gpu:0' , 'PRE_LAYER' ,64)
B=LAYER_2(A    , '/gpu:0' , 'RESIDUAL_1',64)
C=LAYER_2(B    , '/gpu:0' , 'RESIDUAL_2',64)
D=LAYER_2(C    , '/gpu:1' , 'RESIDUAL_3',64)

E=LAYER_2(D    , '/gpu:1' , 'RESIDUAL_4',128,[1,2,2,1])
F=LAYER_2(E    , '/gpu:1' , 'RESIDUAL_5',128)
G=LAYER_2(F    , '/gpu:1' , 'RESIDUAL_6',128)
H=LAYER_2(G    , '/gpu:1' , 'RESIDUAL_7',128)

I=LAYER_2(H  , '/gpu:1' , 'RESIDUAL_8' ,256,[1,2,2,1])
J=LAYER_2(I  , '/gpu:2' , 'RESIDUAL_9' ,256)
K=LAYER_2(J  , '/gpu:2' , 'RESIDUAL_10',256)
L=LAYER_2(K  , '/gpu:2' , 'RESIDUAL_11',256)


M=LAYER_2(L  , '/gpu:3' , 'RESIDUAL_12',512,[1,2,2,1])
N=LAYER_2(M  , '/gpu:3' , 'RESIDUAL_13',512)
O=LAYER_2(N   , '/gpu:3' , 'RESIDUAL_14',512)
P=AVG_POOL(O , ksize=[1,2,2,1] , strides=[1,1,1,1] , pooling='SAME' , layer_name='AVG' ,device_='/gpu:3')

"""
B=STEM_B(A , '/gpu:0')
C=STEM_C(B , '/gpu:0')
D=INCEPTION_MODULE_A(C,'/gpu:0', 'INCEPTION_MODULE_A_1')
E=INCEPTION_MODULE_A(D,'/gpu:0', 'INCEPTION_MODULE_A_2')
F=INCEPTION_MODULE_A(E,'/gpu:0', 'INCEPTION_MODULE_A_3')
G=INCEPTION_MODULE_A(F,'/gpu:0', 'INCEPTION_MODULE_A_4')
H=INCEPTION_REDUCTION_A(G,'/gpu:0' , 'INCEPTION_REDUCTION_A_1')
I=INCEPTION_MODULE_B(H,'/gpu:0' , 'INCEPTION_REDUCTION_B_1')
J=INCEPTION_MODULE_B(I,'/gpu:0' , 'INCEPTION_REDUCTION_B_2')
K=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_3')
L=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_4')
M=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_5')
"""

flat_=FLAT(P)
y_conv=FC_B(flat_ , n_classes ,'/gpu:4' )
#y_conv=FC_A(flat_ , n_classes ,'/gpu:1' ,dropout_rate=keep_prob)
cost , train_step ,correct_prediction , accuracy ,softmax = \
TRAIN_STRUCTURE_A(y_conv , y_ , '/gpu:4' , learning_rate = 0.0001)

tensor_info['cost']=cost
tensor_info['train_step']=train_step
tensor_info['correct_prediction']=correct_prediction
tensor_info['accuracy']=accuracy
tensor_info['softmax']=softmax

place_info['x_']=x_
place_info['y_']=y_
place_info['keep_prob']=keep_prob


logdir='./eye_128/2
sess=START_SESS()
tensorboard_info=OPEN_TENSORBOARD(sess ,tensor_info ,logdir)
fp=make_logdir(dirname)
BATCH_TRAINING_RANDOM(300000 , 30,file_locate , tensor_info,place_info ,fp ,tensorboard_info)
#BATCH_TRAINING_DIVIDE_RANDOM(100000 , 30,file_locate , tensor_info,place_info ,fp)


In [ ]:
tensor_info={};place_info={}
file_locate='./eye_numpy_64/'
dirname='./result_eye/'

A=CONV_1(x_ , '/gpu:1' , 'CONVx5_1',16,(3,3),[1,2,2,1],'SAME')
B=CONV_5(A  , '/gpu:1' , 'CONVx5_2',16,(3,3),[1,1,1,1],'SAME')
C=CONV_5(B  , '/gpu:1' , 'CONVx5_3',16,(3,3),[1,1,1,1],'SAME')
D=CONV_5(C  , '/gpu:1' , 'CONVx5_4',16,(3,3),[1,1,1,1],'SAME')

E=CONV_1(D  , '/gpu:1' , 'CONVx5_5',32,(3,3),[1,2,2,1],'SAME')
F=CONV_5(E  , '/gpu:1' , 'CONVx5_6',32,(3,3),[1,1,1,1],'SAME')
G=CONV_5(F  , '/gpu:1' , 'CONVx5_7',32,(3,3),[1,1,1,1],'SAME')
H=CONV_5(G  , '/gpu:1' , 'CONVx5_8',32,(3,3),[1,1,1,1],'SAME')

I=CONV_1(H  , '/gpu:1' , 'CONVx5_9',64,(3,3),[1,2,2,1],'SAME')
J=CONV_5(I  , '/gpu:1' , 'CONVx5_10',64,(3,3),[1,1,1,1],'SAME')
K=CONV_5(J  , '/gpu:1' , 'CONVx5_11',64,(3,3),[1,1,1,1],'SAME')
L=CONV_5(K  , '/gpu:1' , 'CONVx5_12',64,(3,3),[1,1,1,1],'SAME')


"""
B=STEM_B(A , '/gpu:0')
C=STEM_C(B , '/gpu:0')
D=INCEPTION_MODULE_A(C,'/gpu:0', 'INCEPTION_MODULE_A_1')
E=INCEPTION_MODULE_A(D,'/gpu:0', 'INCEPTION_MODULE_A_2')
F=INCEPTION_MODULE_A(E,'/gpu:0', 'INCEPTION_MODULE_A_3')
G=INCEPTION_MODULE_A(F,'/gpu:0', 'INCEPTION_MODULE_A_4')
H=INCEPTION_REDUCTION_A(G,'/gpu:0' , 'INCEPTION_REDUCTION_A_1')
I=INCEPTION_MODULE_B(H,'/gpu:0' , 'INCEPTION_REDUCTION_B_1')
J=INCEPTION_MODULE_B(I,'/gpu:0' , 'INCEPTION_REDUCTION_B_2')
K=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_3')
L=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_4')
M=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_5')
"""

flat_=FLAT(L)
#y_conv=FC_B(flat_ , 10 ,'/gpu:1' )
y_conv=FC_A(flat_ , n_classes ,'/gpu:1' ,dropout_rate=keep_prob)
cost , train_step ,correct_prediction , accuracy ,softmax = \
TRAIN_STRUCTURE_A(y_conv , y_ , '/gpu:1' , learning_rate = 0.0001)

tensor_info['cost']=cost
tensor_info['train_step']=train_step
tensor_info['correct_prediction']=correct_prediction
tensor_info['accuracy']=accuracy
tensor_info['softmax']=softmax

place_info['x_']=x_
place_info['y_']=y_
place_info['keep_prob']=keep_prob


logdir='./eye_128/1'
sess=START_SESS()
tensorboard_info=OPEN_TENSORBOARD(sess ,tensor_info ,logdir)
fp=make_logdir(dirname)
BATCH_TRAINING_RANDOM(50000 , 30,file_locate , tensor_info,place_info ,fp ,tensorboard_info)
#BATCH_TRAINING_DIVIDE_RANDOM(100000 , 30,file_locate , tensor_info,place_info ,fp)


In [ ]:
sess.close()


In [ ]:
sess._closed

In [ ]:
sess._opened


In [ ]:
tensor_info={};place_info={}
file_locate='./cifar_merge/'
dirname='./result/'

A=CONV_5(x_ , '/gpu:1' , 'CONVx5_1',200,(3,3),[1,1,1,1],'SAME')
B=CONV_5(A , '/gpu:1' , 'CONVx5_2',200,(3,3),[1,1,1,1],'SAME')
"""
B=STEM_B(A , '/gpu:0')
C=STEM_C(B , '/gpu:0')
D=INCEPTION_MODULE_A(C,'/gpu:0', 'INCEPTION_MODULE_A_1')
E=INCEPTION_MODULE_A(D,'/gpu:0', 'INCEPTION_MODULE_A_2')
F=INCEPTION_MODULE_A(E,'/gpu:0', 'INCEPTION_MODULE_A_3')
G=INCEPTION_MODULE_A(F,'/gpu:0', 'INCEPTION_MODULE_A_4')
H=INCEPTION_REDUCTION_A(G,'/gpu:0' , 'INCEPTION_REDUCTION_A_1')
I=INCEPTION_MODULE_B(H,'/gpu:0' , 'INCEPTION_REDUCTION_B_1')
J=INCEPTION_MODULE_B(I,'/gpu:0' , 'INCEPTION_REDUCTION_B_2')
K=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_3')
L=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_4')
M=INCEPTION_MODULE_B(G,'/gpu:0' , 'INCEPTION_REDUCTION_B_5')
"""

flat_=FLAT(B)
y_conv=FC_A(flat_ , n_classes ,'/gpu:1' ,dropout_rate=keep_prob)
tensor_info = TRAIN_STRUCTURE_A(y_conv , y_ , '/gpu:1')

place_info['x_']=x_
place_info['y_']=y_
place_info['keep_prob']=keep_prob


logdir='./module_conv5x2'
sess=START_SESS()
tensorboard_info=OPEN_TENSORBOARD(sess ,tensor_info ,logdir)
fp=make_logdir(dirname)
BATCH_TRAINING_RANDOM(100000 , 30,file_locate , tensor_info,place_info ,fp ,tensorboard_info)
#BATCH_TRAINING_DIVIDE_RANDOM(100000 , 30,file_locate , tensor_info,place_info ,fp)
